In [ ]:
import heapq
import time
import numpy as np

class Puzzle:
    def __init__(self, board, parent=None, move=None, depth=0, cost=0):
        self.board = board
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.empty_tile = self.find_empty_tile()

    def find_empty_tile(self):
        for i in range(4):
            for j in range(4):
                if self.board[i][j] == 0:
                    return (i, j)

    def get_possible_moves(self):
        x, y = self.empty_tile
        moves = []
        directions = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}

        for move, (dx, dy) in directions.items():
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 4 and 0 <= new_y < 4:
                new_board = [row[:] for row in self.board]
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                moves.append(Puzzle(new_board, self, move, self.depth + 1))

        return moves

    def is_goal(self):
        goal = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
        return self.board == goal

    def __lt__(self, other):
        return self.cost < other.cost

def misplaced_tiles_heuristic(state):
    goal = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
    return sum(state.board[i][j] != goal[i][j] and state.board[i][j] != 0 for i in range(4) for j in range(4))

def manhattan_distance_heuristic(state):
    goal_positions = {val: (i, j) for i, row in enumerate([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]) for j, val in enumerate(row)}
    return sum(abs(i - goal_positions[val][0]) + abs(j - goal_positions[val][1]) for i, row in enumerate(state.board) for j, val in enumerate(row) if val != 0)

def a_star(initial_state, heuristic):
    open_list = []
    closed_set = set()
    heapq.heappush(open_list, (0, initial_state))
    explored_nodes = 0

    while open_list:
        _, current = heapq.heappop(open_list)
        explored_nodes += 1

        if current.is_goal():
            path = []
            while current:
                path.append(current)
                current = current.parent
            return path[::-1], explored_nodes

        closed_set.add(tuple(map(tuple, current.board)))

        for neighbor in current.get_possible_moves():
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue

            neighbor.cost = neighbor.depth + heuristic(neighbor)
            heapq.heappush(open_list, (neighbor.cost, neighbor))

    return None, explored_nodes

def print_solution(solution):
    if solution:
        for state in solution:
            for row in state.board:
                print(row)
            print("Move:", state.move, "\n")
    else:
        print("No solution found.")

# Example initial state
initial_board = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]]
initial_state = Puzzle(initial_board)

# Solve using misplaced tiles heuristic
start_time = time.time()
solution_misplaced, nodes_misplaced = a_star(initial_state, misplaced_tiles_heuristic)
misplaced_time = time.time() - start_time

print("Solution using Misplaced Tiles heuristic:")
print_solution(solution_misplaced)
print(f"Nodes explored: {nodes_misplaced}, Time taken: {misplaced_time:.4f} sec\n")

# Solve using Manhattan distance heuristic
start_time = time.time()
solution_manhattan, nodes_manhattan = a_star(initial_state, manhattan_distance_heuristic)
manhattan_time = time.time() - start_time

print("Solution using Manhattan Distance heuristic:")
print_solution(solution_manhattan)
print(f"Nodes explored: {nodes_manhattan}, Time taken: {manhattan_time:.4f} sec\n")

# Compare performance
print("Performance Comparison:")
print(f"Misplaced Tiles - Nodes: {nodes_misplaced}, Time: {misplaced_time:.4f} sec")
print(f"Manhattan Distance - Nodes: {nodes_manhattan}, Time: {manhattan_time:.4f} sec")

Solution using Misplaced Tiles heuristic:
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 0, 12]
[13, 14, 11, 15]
Move: None 

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 0, 15]
Move: Down 

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]
Move: Right 

Nodes explored: 3, Time taken: 0.0002 sec

Solution using Manhattan Distance heuristic:
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 0, 12]
[13, 14, 11, 15]
Move: None 

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 0, 15]
Move: Down 

[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]
Move: Right 

Nodes explored: 3, Time taken: 0.0002 sec

Performance Comparison:
Misplaced Tiles - Nodes: 3, Time: 0.0002 sec
Manhattan Distance - Nodes: 3, Time: 0.0002 sec
